<a href="https://colab.research.google.com/github/TanushGoel/Atari-Games-RL/blob/master/MusicAI_CuDNNLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip3 -q install tensorflow-gpu==1.15.0   # ignore error messages - make sure NOT to use tensorflow 2.0, any tensorflow version between 1.13 and 1.15 will work fine

     |████████████████████████████████| 411.5MB 37kB/s 
     |████████████████████████████████| 3.8MB 37.5MB/s 
     |████████████████████████████████| 512kB 40.2MB/s 
ERROR: tensorflow 2.2.0 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement tensorboard<2.3.0,>=2.2.0, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement tensorflow-estimator<2.3.0,>=2.2.0, but you'll have tensorflow-estimator 1.15.1 which is incompatible.
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.


In [0]:
!pip3 -q install keras

In [0]:
# imports
import music21
from music21 import converter, instrument, note, chord, stream, common
import os
from os import path
import zipfile
import glob
import pickle
import numpy as np
import tensorflow as tf
import keras
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, CuDNNLSTM
from keras import initializers
from keras.callbacks import ModelCheckpoint
from google.colab import files

Using TensorFlow backend.


In [0]:
tf.__version__

'1.15.0'

In [0]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [0]:
tf.keras.__version__

'2.2.4-tf'

In [0]:
# upload zip file first
for i in os.listdir():
  global zip_file, folder
  if zipfile.is_zipfile(i):
      zip_file = str(i)
      folder = zip_file[:-4]
      with zipfile.ZipFile(zip_file, 'r') as zip_ref:
          zip_ref.extractall(os.mkdir(folder))

In [0]:
# one file at a time
notes = []
count = 0
total = len(os.listdir(folder))

for file in glob.glob(folder+"/*.mid"):

    count+=1
    print(f"{count*100/total:1.2f}% Complete")

    try:  
      midi = converter.parse(file)
      parts = instrument.partitionByInstrument(midi)

      if parts: # file has instrument parts
          if len(parts.parts) > 1: # the file has more than one instrument
            print(file, "has more than one instrument")
            continue
          else:
            notes_to_parse = parts.parts[0].recurse()
      else: # file has notes in a flat structure
          notes_to_parse = midi.flat.notes
      for element in notes_to_parse:
          if isinstance(element, note.Note):
              notes.append(str(element.pitch))
          elif isinstance(element, chord.Chord):
              notes.append('.'.join(str(n) for n in element.normalOrder))
  
    except:
      print(file, "could not be parsed")

In [0]:
# first check if gpu use is functional to parse multiple files in parallel
music21.common.parallel.cpus()

3

In [0]:
# multiple files in parallel
filez = []
for file in glob.glob(folder+"/*.mid"):
    filez.append(file)

def get_notes(file):

  notes = []
  notes_to_parse = None
  midi = converter.parse(file)
  parts = instrument.partitionByInstrument(midi)

  try:
    if parts: # file has instrument parts
        if len(parts.parts) > 1: # the file has more than one instrument
          print(file, "has more than one instrument") 
        else:
          notes_to_parse = parts.parts[0].recurse()
    else: # file has notes in a flat structure
        notes_to_parse = midi.flat.notes
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))
  except:
    print(file, "could not be parsed")

  return notes

output = common.runParallel(filez, parallelFunction=get_notes)

notes = [j for i in range(0, len(output)) for j in output[i]]

In [0]:
sequence_length = 100

# get all pitch names
pitchnames = sorted(set(item for item in notes))

# create a dictionary to map pitches to integers
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
network_input = []
network_output = []

# create input sequences and the corresponding outputs
for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])
    
n_patterns = len(network_input)

# reshape the input into a format compatible with CuDNNLSTM layers
network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))

n_vocab = np.amax(network_input)

In [0]:
# normalize input
network_input = np.divide(network_input, float(n_vocab))
network_output = np_utils.to_categorical(network_output)

In [0]:
# CuDDNLSTM model
model = Sequential()
model.add(CuDNNLSTM(256, input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True, kernel_initializer=initializers.RandomNormal(stddev=0.175),
    bias_initializer=initializers.Zeros()))
model.add(Dropout(0.225))
model.add(CuDNNLSTM(512, return_sequences=True, kernel_initializer=initializers.RandomNormal(stddev=0.175)))
model.add(Dropout(0.225))
model.add(CuDNNLSTM(256, kernel_initializer=initializers.RandomNormal(stddev=0.175),
    bias_initializer=initializers.Zeros()))
model.add(Dense(256, kernel_initializer=initializers.RandomNormal(stddev=0.175),
    bias_initializer=initializers.Zeros()))
model.add(Dropout(0.225))
model.add(Dense(n_vocab+1, kernel_initializer=initializers.RandomNormal(stddev=0.25),
    bias_initializer=initializers.Zeros()))
model.add(Activation('softmax'))
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm_1 (CuDNNLSTM)     (None, 125, 256)          265216    
_________________________________________________________________
dropout_1 (Dropout)          (None, 125, 256)          0         
_________________________________________________________________
cu_dnnlstm_2 (CuDNNLSTM)     (None, 125, 512)          1576960   
_________________________________________________________________
dropout_2 (Dropout)          (None, 125, 512)          0         
_________________________________________________________________
cu_dnnlstm_3 (CuDNNLSTM)     (None, 256)               788480    
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
_______________________________________

In [0]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [0]:
# checkpointer
checkpoint = ModelCheckpoint("MusicAI_best.hdf5", 
                             monitor='loss', 
                             verbose=1,        
                             save_best_only=True)

In [0]:
# train model
model.fit(network_input, network_output, epochs=50, batch_size=25, callbacks=[checkpoint])

In [0]:
# make piece
start = np.random.randint(0, len(network_input)-1)
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
pattern = network_input[start]
prediction_output = []

# generate 10000 notes
for note_index in range(10000):
    prediction_input = np.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)
    prediction = model.predict(prediction_input, verbose=0)
    index = np.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)
    pattern = np.append(pattern, index)
    pattern = pattern[1:len(pattern)]

In [0]:
# make piece
offset = 0
output_notes = []
# create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
    # increase offset each iteration so that notes do not stack
    offset += 0.5 # offset += 0.25

In [0]:
# stream piece into midi file
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='output.mid')

'output.mid'

In [0]:
# download piece
files.download('/content/output.mid')